# Week 3 - Segmenting and Clustering Neighborhoods in Toronto


#### Task 1 - Prepare the dataframe

1. import libraries and scrape with beautifulsoup

In [1]:
import requests
import pandas as pd
import numpy as np

In [2]:
! easy_install beautifulsoup4

Searching for beautifulsoup4
Best match: beautifulsoup4 4.6.0
Adding beautifulsoup4 4.6.0 to easy-install.pth file

Using /opt/conda/envs/DSX-Python35/lib/python3.5/site-packages
Processing dependencies for beautifulsoup4
Finished processing dependencies for beautifulsoup4


In [3]:
from bs4 import BeautifulSoup

In [4]:
res = requests.get("https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M")
soup = BeautifulSoup(res.content,'lxml')
table = soup.find_all('table')[0]

In [5]:
codes_list=[]
borough_list=[]
neighborhood_list=[]
i=1
for tag in soup.table.find_all('td'):
    if i == 1:
        codes_list.append(tag.text)
    if i == 2:
        borough_list.append(tag.text)
    if i == 3: 
        neighborhood_list.append(tag.text)
    i = i+1
    if i==4:
        i=1

2. Creating the dataframe

In [6]:
d = {'PostalCode': codes_list, 'Borough': borough_list, 'Neighborhood': neighborhood_list}
tor = pd.DataFrame(data=d,columns=['PostalCode','Borough','Neighborhood'])

3. Cleaning the dataframe: removing "Not assigned" under Borough and remove "\n" values in the Neighbourhood names

In [7]:
tor_b = tor[tor.Borough != 'Not assigned']

In [8]:
tor_b = tor_b.copy()
tor_b['Neighborhood'].replace(to_replace='\n', value='', regex=True, inplace=True)

4. Cleaning the dataframe: replacing "Not assigned" under Neighborhood with the Borough name

In [9]:
tor_b['Neighborhood'].replace(to_replace='Not assigned', value=tor_b['Borough'], inplace=True)

5. Combining Neighborhood names with same Postal Code and separating with ,

In [10]:
tor_j = tor_b.groupby(['PostalCode','Borough'], as_index=False, sort=False).agg(lambda x: ', '.join(x))
tor_j.head()

,PostalCode,Borough,Neighborhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,"Harbourfront, Regent Park"
3,M6A,North York,"Lawrence Heights, Lawrence Manor"
4,M7A,Queen's Park,Queen's Park


6. Shape output of dataframe

In [11]:
print('The dataframe has {} boroughs and {} neighborhoods.'.format(
        len(tor_j['Borough'].unique()),
        tor_j.shape[0]))

The dataframe has 11 boroughs and 103 neighborhoods.


In [12]:
tor_j.shape[0]

103

#### Task 2 - Add coordinates

1. Read csv file (geocoder did not collect enough data)

In [13]:
coor = pd.read_csv('http://cocl.us/Geospatial_data', sep=',')

2. Setting indexes as preparation for concat function

In [14]:
coor.rename(columns={'Postal Code':'PostalCode'}, inplace=True)
coor.set_index('PostalCode', inplace=True)

In [15]:
tor_j.set_index('PostalCode', inplace=True)

3. Applying concat function to join both dataframes and reset the index

In [17]:
tor_coor = pd.concat([tor_j, coor], axis=1)
tor_coor.reset_index(inplace=True)
tor_coor.head()

,index,Borough,Neighborhood,Latitude,Longitude
0,M1B,Scarborough,"Rouge, Malvern",43.806686,-79.194353
1,M1C,Scarborough,"Highland Creek, Rouge Hill, Port Union",43.784535,-79.160497
2,M1E,Scarborough,"Guildwood, Morningside, West Hill",43.763573,-79.188711
3,M1G,Scarborough,Woburn,43.770992,-79.216917
4,M1H,Scarborough,Cedarbrae,43.773136,-79.239476
